# SignifyPy Randy Keys
This notebook should help a developer validate their env setup (Witnesses, KERIA, and SignifyPy client) and walk them step-by-step through the randy key creation. If you haven't setup and run KERI/Witnesses before [Kent Bull's KERI/ACDC guide](https://kentbull.com/2023/03/09/keri-tutorial-series-treasure-hunting-in-abydos-issuing-and-verifying-a-credential-acdc/) is a great starting point

## Env Setup

### Witness Network
Run a **clean** local witness network
* Clear out any of the local files from previous runs. Note keri wants to write to /user/local/var/keri, or ~/.keri
* And launch the local witness network
* You will know your witness network isn't clean if you see a KERIA error like:
``` ValueError: Already incepted pre=DCQE55HKUokSAKc4ntJw2b845r4DnddpPZs8iF_T-0c-. ERR: mdb_txn_renew: MDB_BAD_RSLOT: Invalid reuse of reader locktable slot```

Start your clean demo witness network with a command like:
```rm -rf /usr/local/var/keri/*;rm -Rf ~/.keri/*;nohup kli witness demo```

## KERIA

Next, start the KERI Agent (KERIA)

```keria start --config-file demo-witness-oobis.json --config-dir ./scripts``` 


## Signify
* If signifypy isn't on PiPy yet, then make sure your Jupyter Python version points to the same Python version where you install signifypy.
* One way to install signifypy is from git like: ```%python -m pip install "signifypy @ git+https://github.com/WebOfTrust/signifypy.git"```

## Install python dependencies from PiPy

In [1]:
!which python
%python -m pip install keri
%python -m pip install responses
%python -m pip install pytest

/Users/meenyleeny/.pyenv/versions/3.11.3/bin/python


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [2]:
from time import sleep

import requests
from keri.app.keeping import Algos
from keri.core import coring
from responses import _recorder

import pytest
from keri import kering
from keri.core.coring import Tiers, Serder, MtrDex

from signify.app.clienting import SignifyClient

## Basic Signify client creation
* Create client using the admin url
* Hard-code the randomness for testing purposes
* Don't set the tier, to create the least expensive/secure public key

In [3]:
admin_url = "http://localhost:3901"
bran = b'0123456789abcdefghijk'
tier = None

# Try with bran that is too short
with pytest.raises(kering.ConfigurationError):
    SignifyClient(url=admin_url, bran=bran[:16], tier=tier)

# Try with an invalid URL
with pytest.raises(kering.ConfigurationError):
    SignifyClient(url="ftp://www.example.com", bran=bran, tier=tier)

client = SignifyClient(url=admin_url, bran=bran, tier=tier)
# Your client public key
assert client.controller == "ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJXJHtJose"
# Get the inception event
serder = client.icp
# Review the inception event
assert serder.raw == (b'{"v":"KERI10JSON00012b_","t":"icp","d":"ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJ'
                        b'XJHtJose","i":"ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJXJHtJose","s":"0","kt":"1'
                        b'","k":["DAbWjobbaLqRB94KiAutAHb_qzPpOHm3LURA_ksxetVc"],"nt":"1","n":["EIFG_u'
                        b'qfr1yN560LoHYHfvPAhxQ5sN6xZZT_E3h7d2tL"],"bt":"0","b":[],"c":[],"a":[]}')
print("Inception event is: ", serder.raw)

Inception event is:  b'{"v":"KERI10JSON00012b_","t":"icp","d":"ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJXJHtJose","i":"ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJXJHtJose","s":"0","kt":"1","k":["DAbWjobbaLqRB94KiAutAHb_qzPpOHm3LURA_ksxetVc"],"nt":"1","n":["EIFG_uqfr1yN560LoHYHfvPAhxQ5sN6xZZT_E3h7d2tL"],"bt":"0","b":[],"c":[],"a":[]}'


* For more information about keri fields, see [that section of the KERI whitepaper](https://weboftrust.github.io/ietf-keri/draft-ssmith-keri.html#section-5.1)

## You can create tiers of clients
The higher tiers take longer to generate as the number of hash iterations increases

In [4]:
tier = Tiers.low
client = SignifyClient(url=admin_url, bran=bran, tier=tier)
print("Low tier controller is: ", client.controller)
assert client.controller == "ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJXJHtJose"

#tier = Tiers.med
#client = SignifyClient(url=admin_url, bran=bran, tier=tier)
#print("Medium tier controller is: ", client.controller)
#assert client.controller == "EOgQvKz8ziRn7FdR_ebwK9BkaVOnGeXQOJ87N6hMLrK0"

#tier = Tiers.high
#client = SignifyClient(url=admin_url, bran=bran, tier=tier)
#print("High tier controller is: ", client.controller)
#assert client.controller == "EB8wN2c_tv1WlsJ5c3949-TFWPMB2IflFbdMlZfC_Hgo"

Low tier controller is:  ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJXJHtJose


## Post inception event to KERIA (OOB) boot interface

In [5]:
evt, siger = client.ctrl.event()
res = requests.post(url="http://localhost:3903/boot",
                    json=dict(
                        icp=evt.ked,
                        sig=siger.qb64,
                        stem=client.ctrl.stem,
                        pidx=1,
                        tier=client.ctrl.tier))
if res.status_code != requests.codes.accepted:
    raise kering.AuthNError(f"unable to initialize cloud agent connection, {res.status_code}, {res.text}")

## Create a KERIA agent session

In [6]:
client.connect()
assert client.agent is not None
assert client.agent.anchor == "ELI7pg979AdhmvrjDeam2eAO2SR5niCgnjAJXJHtJose"
assert client.agent.pre == "EJoqUMpQAfqsJhBqv02ehR-9BJYBTCrW8h5JlLdMTWBg"
assert client.ctrl.ridx == 0

## Create a KERI Autonomic Identifier (AID)

In [7]:
identifiers = client.identifiers()
aids = identifiers.list()
assert aids == []

aid = identifiers.create("aid1")
icp = Serder(ked=aid)
assert icp.pre == "ELUvZ8aJEHAQE-0nsevyYTP98rBbGJUrTj5an-pCmwrK"
assert len(icp.verfers) == 1
assert icp.verfers[0].qb64 == "DPmhSfdhCPxr3EqjxzEtF8TVy0YX7ATo0Uc8oo2cnmY9"
assert len(icp.digers) == 1
assert icp.digers[0].qb64 == "EAORnRtObOgNiOlMolji-KijC_isa3lRDpHCsol79cOc"
assert icp.tholder.num == 1
assert icp.ntholder.num == 1

rpy = identifiers.makeEndRole(pre=icp.pre, eid="EPGaq6inGxOx-VVVEcUb_KstzJZldHJvVsHqD4IPxTWf")

aids = identifiers.list()
assert len(aids) == 1
aid = aids.pop()

salt = aid[Algos.salty]
assert aid['name'] == "aid1"
assert salt["pidx"] == 0
assert aid["prefix"] == icp.pre
assert salt["stem"] == "signify:aid"

## Create a second AID with more configuration

In [8]:
aid2 = identifiers.create("aid2", count=3, ncount=3, isith="2", nsith="2")
icp2 = Serder(ked=aid2)
assert icp2.pre == "EI5e4q43vsTsy-vJFcVGKfI3YKHbOT5ffuseaxtuYydL"
assert len(icp2.verfers) == 3
assert icp2.verfers[0].qb64 == "DPmhSfdhCPxr3EqjxzEtF8TVy0YX7ATo0Uc8oo2cnmY9"
assert icp2.verfers[1].qb64 == "DHgomzINlGJHr-XP3sv2ZcR9QsIEYS3LJhs4KRaZYKly"
assert icp2.verfers[2].qb64 == "DEfdjYZMI2hLaHBOpUubn5AUItgOvh2W1vckGE33SIPf"
assert len(icp2.digers) == 3
assert icp2.digers[0].qb64 == "EEvyqpRLktts-_aSfPHKKv1mTKTV4ngwKKkOaqm3ZuPX"
assert icp2.digers[1].qb64 == "EEkMimwsv_JMZh7k-Rfq5wvhvbEdjVr8NhGQpyssVmNJ"
assert icp2.digers[2].qb64 == "EJy_MjjMWLJkn_5cRaUtDr7asfLe70xbAPD2nablr0iv"
assert icp2.tholder.num == 2
assert icp2.ntholder.num == 2

aids = identifiers.list()
assert len(aids) == 2
aid = aids[1]
assert aid['name'] == "aid2"
assert aid["prefix"] == icp2.pre
salt = aid[Algos.salty]
assert salt["pidx"] == 1
assert salt["stem"] == "signify:aid"

ked = identifiers.rotate("aid1")
rot = Serder(ked=ked)

assert rot.said == "EBQABdRgaxJONrSLcgrdtbASflkvLxJkiDO0H-XmuhGg"
assert rot.sn == 1
assert len(rot.digers) == 1
assert rot.verfers[0].qb64 == "DHgomzINlGJHr-XP3sv2ZcR9QsIEYS3LJhs4KRaZYKly"
assert rot.digers[0].qb64 == "EJMovBlrBuD6BVeUsGSxLjczbLEbZU9YnTSud9K4nVzk"

## Create an interaction event

In [9]:
ked = identifiers.interact("aid1", data=[icp.pre])
ixn = Serder(ked=ked)
assert ixn.said == "ENsmRAg_oM7Hl1S-GTRMA7s4y760lQMjzl0aqOQ2iTce"
assert ixn.sn == 2
assert ixn.ked["a"] == [icp.pre]

aid = identifiers.get("aid1")
state = aid["state"]
assert state['s'] == '2'
assert state['f'] == '2'
assert state['et'] == 'ixn'
assert state['d'] == ixn.said
assert state['ee']['d'] == rot.said

## Review the key events and list the identifiers

In [10]:
events = client.keyEvents()
log = events.get(pre=aid["prefix"])
assert len(log) == 3
serder = coring.Serder(ked=log[0])
assert serder.pre == icp.pre
assert serder.said == icp.said
serder = coring.Serder(ked=log[1])
assert serder.pre == rot.pre
assert serder.said == rot.said
serder = coring.Serder(ked=log[2])
assert serder.pre == ixn.pre
assert serder.said == ixn.said

print(identifiers.list())

[{'name': 'aid1', 'prefix': 'ELUvZ8aJEHAQE-0nsevyYTP98rBbGJUrTj5an-pCmwrK', 'salty': {'pidx': 0, 'kidx': 1, 'stem': 'signify:aid', 'tier': 'low', 'dcode': 'E', 'icodes': ['A'], 'ncodes': ['A'], 'transferable': True}}, {'name': 'aid2', 'prefix': 'EI5e4q43vsTsy-vJFcVGKfI3YKHbOT5ffuseaxtuYydL', 'salty': {'pidx': 1, 'kidx': 0, 'stem': 'signify:aid', 'tier': 'low', 'dcode': 'E', 'icodes': ['A', 'A', 'A'], 'ncodes': ['A', 'A', 'A'], 'transferable': True}}]
